In [10]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset('sst', trust_remote_code=True)

Generating test split: 100%|██████████| 2210/2210 [00:03<00:00, 597.72 examples/s] 


In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 2210
    })
})

In [17]:
# Read and map the scores to their classes
def score_to_class(score):
    if score <= 0.2:
        return 0
    elif score <= 0.4:
        return 1
    elif score <= 0.6:
        return 2
    elif score <= 0.8:
        return 3
    else:
        return 4

train_data = dataset['train'].to_pandas()[['sentence', 'label']]
train_data['sentence'] = train_data['sentence'].apply(lambda x: x.lower())
train_data['label'] = train_data['label'].apply(lambda x: score_to_class(x))

test_data = dataset['test'].to_pandas()[['sentence', 'label']]
test_data['sentence'] = test_data['sentence'].apply(lambda x: x.lower())
test_data['label'] = test_data['label'].apply(lambda x: score_to_class(x))

val_data = dataset['validation'].to_pandas()[['sentence', 'label']]
val_data['sentence'] = val_data['sentence'].apply(lambda x: x.lower())
val_data['label'] = val_data['label'].apply(lambda x: score_to_class(x))

In [30]:
import numpy as np
np.sum(train_data['label'] == 2)

1624